# 1. Librería requests

Esta librería nos va a permitir simular peticiones web y poder trabajar con ellas simulando nuestros "clicks" en el ordenador o incluso usar una API.

Para importarla haremos un `import requests`.

Las llamadas tipicas son: `requests.get()` y `requests.post()`.
También podemos usar otra llamada generica que sea `requests.Request('METODO')`.

Otra opcion es crear una "session" la cual nos mantendrá las cookies y demás caracteristicas de las peticiones, como tokens especiales y similares, para ello haremos uso de `requests.Session()`.

Los códigos de estado nos dicen si la petición ha fallado, ha funcionado o si el servidor ha fallado.

Estos codigos son normalmente genéricos, por ejemplo el típico 404 significa no se ha encontrado.

Cuando queremos leer texto en plano usaremos `.text`, sin embargo, los datos son un diccionario usaremos `.json()`

In [22]:
# Vamos a realizar un programa que importe requests y vamos a recibir nuestro primer hello world
import requests
headers = {'Accept': 'application/json'}
r = requests.get('https://sandbox.api.service.nhs.uk/hello-world/hello/world',headers=headers)
# Eso nos va a devolver en r todo el estado de la peticion y usaremos las siguientes funciones:

print(r.text) #esto es la peticion tal cual la recibes en texto plano
variable = r.json() # En caso de que sea un diccionario lo recogeremos así
# La diferencia es la siguiente, podemos hacer un r.json() a una variable y leer el campo message
print(variable['message'])

{
  "message": "Hello World!"
}

Hello World!


# 1.1 Llamadas a APIs:

Vamos a sacar el tiempo en nuestra provincia (Navarra)

Mas info: https://www.el-tiempo.net/api


In [103]:
URL_BASE = 'https://www.el-tiempo.net/api/json/v2/'

r1 = requests.get(URL_BASE + 'provincias/31').json()
# Comprobamos que valores tiene:
print(r1.keys())
# imprimimos el resumen de hoy
print(r1['today'])

dict_keys(['title', 'today', 'tomorrow', 'ciudades', 'provincia', 'comautonoma', 'breadcrumb', 'metadescripcion', 'keywords'])
{'p': 'Nuboso o cubierto disminuyendo a intervalos nubosos a partir del mediodía, excepto en el tercio sur, donde se esperan cielos poco nubosos. Lluvias débiles y algún chubasco ocasional en Pirineos y noroeste de madrugada. Temperaturas mínimas en ligero ascenso, más acusado en la Ribera, y máximas en descenso, que será notable en el tercio norte y ligero o sin cambios en el tercio sur. Las mínimas tendrán lugar al final del día en muchas zonas. Heladas débiles dispersas en el extremo occidental y en Pirineos. Viento flojo de del noroeste, algo más intenso en la Ribera.'}


Conseguid el tiempo en pamplona usando la API Postdata: la ID del municipio son los primeros 5 caracteres de CODIGOINE, tendréis que buscarlo vosotros :)

In [23]:
## RELLENAR
URL_BASE = 'https://www.el-tiempo.net/api/json/v2/'
ID='31015'
CODPROV=ID[:2]

r2=requests.get(URL_BASE+'provincias/'+CODPROV+'/municipios/'+ID)
if r.ok: print('Datos recividos')
else: print('Error en la transmision de datos')
rj2=r2.json()
print(rj2.keys())


Datos recividos
dict_keys(['municipio', 'fecha', 'stateSky', 'temperatura_actual', 'temperaturas', 'humedad', 'viento', 'lluvia', 'imagen', 'pronostico', 'proximos_dias', 'breadcrumb', 'keywords'])


In [109]:
#for i in rj:
#    print(i.capitalize()+":",rj[i])
#rj2['pronostico']['hoy'].keys() #Poniendo rj['ATRIBUTO'] con ATRIBUTO lo que quieras saber te devuelve esa informacion
rj2['pronostico']['manana'].keys()

dict_keys(['@attributes', 'estado_cielo', 'precipitacion', 'prob_precipitacion', 'prob_tormenta', 'nieve', 'prob_nieve', 'temperatura', 'sens_termica', 'humedad_relativa', 'viento', 'racha_max'])

In [101]:
rj2['proximos_dias'][0].keys()

dict_keys(['@attributes', 'prob_precipitacion', 'cota_nieve_prov', 'estado_cielo', 'viento', 'racha_max', 'temperatura', 'sens_termica', 'humedad_relativa', 'uv_max'])

# 1.2 Intentad engañar al servidor

Python nos permite modificar toda la petición web, intentad engañar al servidor diciendole que estais usando un `User-Agent` llamado `Administrador`

In [5]:
## RELLENAR
URL_BASE = 'https://www.el-tiempo.net/api/json/v2/'
headers={'User-Agent':'Administrador'}

r=requests.get(URL_BASE+'provincias/'+CODPROV+'/municipios/'+ID,headers=headers)
print(r.ok)
rj=r.json()
print(rj.keys()==rj2.keys()) #La API admite el user-agent "Administrador", pero no veo ninguna diferencia en la información que nos devuelve

True
True


# 2. Decoradores de python

Los decoradores son funciones que toman otra funcion como argumento y devuelven otra funcion. Un jaleo si...

Pero resulta que pueden llegar a ser muy utiles en un futuro. Veamos un pequeño ejemplo


In [6]:
# Creamos una funcion que sume dos numeros
# Resulta que queremos meterle una lista de listas de numeros en vez de solo dos valores.
# Para ello creamos otra funcion que tome como argumento sumar y lo implemente.

def decorador_sumar(funcion):
    def interna(lista_de_valores):
        devolver = []
        for i in lista_de_valores:
            devolver.append(funcion(i[0],i[1]))
        return devolver
    return interna

@decorador_sumar
def sumar(a,b):
    return a+b

listado = [(1,2),(3,4),(5,6),(7,8)]

print(sumar(listado))
# sumar = decorador_sumar(sumar)


[3, 7, 11, 15]


# 2.1 Ejercicio

Cread una funcion que eleve un numero con otro y usad el decorador para poder lanzarle una lista de tuplas. A ver si conseguis cambiar el codigo del for por un generador :)

In [7]:
## Rellenar
@decorador_sumar
def elevar(a,b):
    return a**b

print(elevar(listado))
#Este método funciona bien. Usando el decorador con el bucle for que hemos definido antes

[1, 81, 15625, 5764801]


In [10]:
#Ahora vamos a intentarlo mediante un generador
#Para ello vamos a definir otro decorador diferente
def decorador_elevar(funcion):
    def interna(listado_de_valores):
        def generar(): #Este será el generador que sustituye al for
            i=1
            for i in listado_de_valores:
                yield funcion(i[0],i[1])
        devuelve=[]
        for x in generar(): #En este bucle for utilizamos el generador para generar la lista de resultados
            devuelve.append(x)
        return devuelve
    return interna

#Utilizamos el decorador nuevo para aplicarlo a la funcion que nos interesa
@decorador_elevar
def elevar2(a,b):
    return a**b

#Probamos si la funcion+decorador funciona mediante un generador
listado = [(8,2),(3,7),(5,5),(7,9)]
print(elevar2(listado))

#Por lo visto funciona, pero no estoy seguro de que sea más eficiente que la versión anterior.
#Por lo pronto es una forma más complicada de hacer lo mismo, ya que tienes que usar un bucle for igualmente.

[64, 2187, 3125, 40353607]


Existe otro decorador muy curioso, que ya esta implementado de serie en `functools` de python, se llama `lru_cache`. Este decorador nos permite implementar funciones recursivas con cache para evitar calcularlas dos veces!

In [2]:
# codigo de ejemplo:

def memoria(funcion):
    memoria = {}
    def ayudante(x):
        if x not in memoria:
            memoria[x] = funcion(x)
        return memoria[x]
    return ayudante

@memoria
def fibonnaci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonnaci(n-1) + fibonnaci(n-2)

print(fibonnaci(40))

102334155


In [12]:
# codigo usando el built in Memoize
import functools

# El max size es el maximo numero de elementos en memoria
@functools.lru_cache(maxsize=None)
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

    
valor=fib(200)
print(valor,fib.cache_info())

280571172992510140037611932413038677189525 CacheInfo(hits=198, misses=201, maxsize=None, currsize=201)


In [26]:
# Ejercicio -> Cread lo una funcion recursiva de un numero factorial
@functools.lru_cache(maxsize=30)
def factorial(n):
    if n<=1:
        return 1
    else:
        return factorial(n-1)*n

valores=[factorial(i) for i in range(20)]
print(valores,factorial.cache_info())

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800, 39916800, 479001600, 6227020800, 87178291200, 1307674368000, 20922789888000, 355687428096000, 6402373705728000, 121645100408832000] CacheInfo(hits=18, misses=20, maxsize=30, currsize=20)


[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800, 39916800, 479001600, 6227020800, 87178291200, 1307674368000, 20922789888000, 355687428096000, 6402373705728000, 121645100408832000]
CacheInfo(hits=18, misses=20, maxsize=None, currsize=20)